In [10]:
!pip install -q pyinterval
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_midpoint(interv):
    return (interv[0].sup + interv[0].inf) / 2

def interval_to_str(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

In [11]:
def moore_tbl(f, df, start, end, tol=1e-6, iteration=1):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'Interval', 'Width', 'Function extension'])

    def tbl_put(iteration, status, X, X_width, f_X):
        tbl.loc[len(tbl)] = [iteration, status, interval_to_str(X), X_width, interval_to_str(f_X)]

    def moore(start, end, iteration=1):
        X = interval([start, end])
        X_width = calculate_width(X)
        f_X = f(X)

        # Stop if no 0 in F(X)
        if 0 not in f_X:
            tbl_put(iteration, 'no root', X, X_width, f_X)
            return
        
        # Return result if tolerance is satisfied
        if X_width < tol:
            tbl_put(iteration, 'RESULT', X, X_width, f_X)
            return
        
        # [Step 2]: Half if 0 in F'(X)
        x_mid = calculate_midpoint(X)
        df_X = df(X)
        if 0 in df_X:
            tbl_put(iteration, 'split', X, X_width, f_X)
            moore(start, x_mid, iteration + 1)
            moore(x_mid,   end, iteration + 1)
            return
        
        # [Step 3]: Stop if X_next is empty
        f_mid = f(x_mid)
        U = x_mid - f_mid / df_X
        X_next = U & X
        if not X_next:
            tbl_put(iteration, 'empty intersection', X, X_width, f_X)
            return

        # [Step 4]: Continue with narrowed interval
        tbl_put(iteration, 'continue', X, X_width, f_X)
        moore(X_next[0].inf, X_next[0].sup, iteration + 1)

    moore(start, end)
    return tbl.sort_values(['Depth', 'Status', 'Interval'])

In [12]:
func = lambda x: imath.sin(x-1)*(x-2)*(x-3)
dfunc = lambda x: (x**2-5*x+6)*imath.cos(x-1)-(5-2*x)*imath.sin(x-1)

moore_tbl(func, dfunc, 0.9, 2.3)

,Depth,Status,Interval,Width,Function extension
0,1,split,"[0.9000000, 2.3000000]",1.400000e+00,"[-0.6070417, 2.2258194]"
1,2,split,"[0.9000000, 1.6000000]",7.000000e-01,"[-0.2306152, 1.3043241]"
9,2,split,"[1.6000000, 2.3000000]",7.000000e-01,"[-0.4046944, 0.5395926]"
8,3,no root,"[1.2500000, 1.6000000]",3.500000e-01,"[0.1385462, 0.7410932]"
10,3,no root,"[1.6000000, 1.9500000]",3.500000e-01,"[0.0296437, 0.4555127]"
2,3,split,"[0.9000000, 1.2500000]",3.500000e-01,"[-0.2306152, 0.5715031]"
11,3,split,"[1.9500000, 2.3000000]",3.500000e-01,"[-0.3035208, 0.0505868]"
3,4,continue,"[0.9000000, 1.0750000]",1.750000e-01,"[-0.2306152, 0.1730876]"
12,4,continue,"[1.9500000, 2.1250000]",1.750000e-01,"[-0.1184226, 0.0473690]"
7,4,no root,"[1.0750000, 1.2500000]",1.750000e-01,"[0.0983452, 0.4405337]"
